In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

# Can the history of Abos be defined?

In [2]:
import ast
import pandas as pd
import numpy as np

# Load Data

In [3]:
bq = pd.read_csv("SprintStories/EN-3061-Lillydo-Abohistorie/bigquery_data.csv")
features = pd.read_csv("SprintStories/EN-3061-Lillydo-Abohistorie/lillydoo_features.csv")

# Check Big Query Events for Abo checkout or cancellations

27.01.2026 - property keys
['custom_conversion_attribute_conversions',
 'custom_conversion_attribute_returning_diaper_subscriptions', **
 'custom_conversion_attribute_shopify_order_product_tags',
 'custom_conversion_attribute_shopify_order_id',
 'custom_conversion_attribute_orders',
 'custom_conversion_attribute_shopify_order_shipping_country_code',
 'custom_conversion_attribute_new_wipes_subscriptions', **
 'custom_conversion_attribute_product_variant_title',
 'custom_conversion_attribute_sold_quantity',
 'custom_conversion_attribute_returning_wipes_subscriptions', **
 'custom_conversion_attribute_conversion_name',
 'custom_conversion_attribute_order_taxes',
 'custom_conversion_attribute_new_gross_revenue',
 'custom_conversion_attribute_new_orders',
 'custom_conversion_attribute_all_conversions',
 'custom_conversion_attribute_new_net_revenue',
 'custom_conversion_attribute_repeat_net_revenue',
 'custom_conversion_attribute_klaviyo_list_name',
 'custom_conversion_attribute_repeat_orders',
 'custom_conversion_attribute_subscription_entry_type', **
 'custom_conversion_attribute_new_diaper_subscriptions', **
 'custom_conversion_attribute_net_revenue',
 'custom_conversion_attribute_gross_revenue',
 'custom_conversion_attribute_net_sales',
 'custom_conversion_attribute_repeat_gross_revenue',
 'custom_conversion_attribute_gross_discounts',
 'custom_conversion_attribute_sales_type',
 'custom_conversion_attribute_product_title',
 'custom_conversion_attribute_subscription_segment',
 'custom_conversion_attribute_gmv',
 'custom_conversion_attribute_refund_taxes',
 'custom_conversion_attribute_gross_refunds',
 'custom_conversion_attribute_discount_code',
 'custom_conversion_attribute_shopify_order_sales_channel']

In [4]:
property_keys = []
for irow, row in bq.iterrows():
    properties = row["properties"]
    if isinstance(properties, str):
        properties = ast.literal_eval(properties)
    property_keys+=list(properties.keys())
    property_keys = list(set(property_keys))
len(property_keys)

In [6]:
bq = bq.drop_duplicates(subset=["_id","created","name","profileId","properties","type","anonymousId"],keep="first")

In [7]:
def return_property_value(entry, key):
    if isinstance(entry, str):
        entry = ast.literal_eval(entry)
    if key in entry.keys():
        return entry[key]
    return None
relevant_property_keys = [
    'custom_conversion_attribute_new_wipes_subscriptions',
    'custom_conversion_attribute_returning_wipes_subscriptions',
    'custom_conversion_attribute_new_diaper_subscriptions',
    'custom_conversion_attribute_returning_diaper_subscriptions',
    'custom_conversion_attribute_subscription_entry_type',
    ]
for property in relevant_property_keys: 
    bq[property] = bq["properties"].apply(lambda x: return_property_value(x, property))
bq

In [10]:
bq["month"] = pd.to_datetime(bq["created"]).dt.month
bq

In [17]:
continous_profiles = bq.groupby("profileId")["month"].nunique().sort_values(ascending=False)
continous_profiles = continous_profiles[continous_profiles > 1]
continous_profiles

In [ ]:
bq_filtered = bq[bq["profileId"].isin(continous_profiles.index)].sort_values(by=["profileId","created"])[["created","profileId","properties"]+relevant_property_keys]
bq_filtered.transpose()

In [ ]:
profiles_cancelled_diaper_subscriptions = bq_filtered[["created","profileId"]+["custom_conversion_attribute_new_diaper_subscriptions","custom_conversion_attribute_returning_diaper_subscriptions"]].copy()
profiles_cancelled_diaper_subscriptions[["custom_conversion_attribute_new_diaper_subscriptions","custom_conversion_attribute_returning_diaper_subscriptions"]] = profiles_cancelled_diaper_subscriptions[["custom_conversion_attribute_new_diaper_subscriptions","custom_conversion_attribute_returning_diaper_subscriptions"]].astype("float")
profiles_cancelled_diaper_subscriptions["new"] = profiles_cancelled_diaper_subscriptions.groupby("profileId")["custom_conversion_attribute_new_diaper_subscriptions"].transform("sum")
profiles_cancelled_diaper_subscriptions["returning"] = profiles_cancelled_diaper_subscriptions.groupby("profileId")["custom_conversion_attribute_returning_diaper_subscriptions"].transform("sum")
profiles_cancelled_diaper_subscriptions

In [49]:
profile_id_cancelled = "f7b7c61a-5e1b-4146-b1b6-8e45ef17cc15"
profile_id_returning = "fe3ac498-fbdc-4d6a-93cf-cd20cc82e325"
bq_filtered[bq_filtered["profileId"] == profile_id_returning].transpose()


In [21]:
import sys
sys.exit()

# Merge with features

In [28]:
merged = pd.merge(features, bq, left_on="bigquery_conversion_id", right_on="_id", how="left", suffixes=("","_bigquery"))
checkout_completed=merged[merged["name"] == "checkout_completed"]["session"].unique()
merged["name"] = np.where(merged["session"].isin(checkout_completed), "checkout_completed", merged["name"])
merged

## Wipes History

In [43]:
wipes = merged[(merged["custom_conversion_attribute_new_wipes_subscriptions"].notna())|(merged["custom_conversion_attribute_returning_wipes_subscriptions"].notna())]
continous_profiles_wipes = wipes.groupby("profileId")["month"].nunique().sort_values(ascending=False)
continous_profiles_wipes = continous_profiles_wipes[continous_profiles_wipes > 1]
print(continous_profiles_wipes)
wipes[wipes["profileId"].isin(continous_profiles_wipes.index)][["created","profileId"]+relevant_property_keys]

## Diapers History

In [42]:
diapers = merged[(merged["custom_conversion_attribute_new_diaper_subscriptions"].notna())|(merged["custom_conversion_attribute_returning_diaper_subscriptions"].notna())]
continous_profiles_diapers = diapers.groupby("profileId")["month"].nunique().sort_values(ascending=False)
continous_profiles_diapers = continous_profiles_diapers[continous_profiles_diapers > 1]
print(continous_profiles_diapers)
diapers[diapers["profileId"].isin(continous_profiles_diapers.index)][["created","profileId"]+relevant_property_keys]